In [1]:
import numpy as np
from bs4 import BeautifulSoup
from scraperFunctions import vtiScraper,vxusScraper, bndScraper, bndxScraper
import pandas as pd
from selenium import webdriver

In [2]:
# Running scrapers
vti_MC_data = vtiScraper()
vxus_MC_data = vxusScraper()
bnd_MC_data = bndScraper()
bndx_MC_data = bndxScraper()
# Saving data
vti_MC_data.to_csv('vti_MC_data.csv',index=False)
vxus_MC_data.to_csv('vxus_MC_data.csv',index=False)
bnd_MC_data.to_csv('bnd_MC_data.csv',index=False)
bndx_MC_data.to_csv('bndx_MC_data.csv',index=False)

NameError: name 'PdfFileReader' is not defined

In [ ]:
# Updating data (If already have updated historical data, go to reader)

In [ ]:
# Reading saved data
vti_MC_data = pd.read_csv('vti_MC_Data.csv')
vxus_MC_data = pd.read_csv('vxus_MC_Data.csv')
bnd_MC_data = pd.read_csv('bnd_MC_Data.csv')
bndx_MC_data = pd.read_csv('bndx_MC_Data.csv')

In [2]:
# Getting current prices
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
etfs = ['vti','bnd','vxus','bndx']
prices = []
for etf in etfs:
    driver.get('https://finance.yahoo.com/quote/'+etf)
    html = driver.page_source
    soup = BeautifulSoup(html,features="html.parser")
    x = soup.find('span', {'class':'Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(ib)'})
    price = float(x.text)
    prices.append(price)
    print(etf,price)
driver.close()

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [22]:
# format [U.S. Stocks, U.S. Bonds, International Stocks, International Bonds] (in trillions for market cap)
# As of December 30,2019
EOQmarketcaps = np.array([31.780092,22.5263,25.734560,19.55274])
EOQprices = np.array([163.62,83.86,55.69,56.58])
currentPrices = np.asarray(prices)
amountInvested = 5925

sum(EOQmarketcaps)

99.593692

In [23]:
EOQproportions = EOQmarketcaps/sum(EOQmarketcaps)
EOQproportions

array([0.31909744, 0.226182  , 0.25839548, 0.19632508])

In [24]:
priceRatios = currentPrices / EOQprices
priceRatios

array([0.97347513, 1.02718817, 0.93050817, 1.02509721])

In [25]:
products = EOQproportions * priceRatios
products

array([0.31063342, 0.23233147, 0.24043911, 0.2012523 ])

In [26]:
# Weights for each etf theoretical
revisedProportions = products / sum(products)
revisedProportions

array([0.31547396, 0.23595185, 0.24418582, 0.20438837])

In [27]:
sum(revisedProportions)

1.0

In [28]:
desiredValues = revisedProportions * amountInvested
desiredValues

array([1869.18320855, 1398.01469089, 1446.80099968, 1211.00110088])

In [29]:
sum(desiredValues)

5925.0

In [30]:
# Shares of etfs to buy theoretically
shares = desiredValues/currentPrices
shares

array([11.73520347, 16.22956456, 27.91974141, 20.87932933])

In [56]:
schwab_shares = shares * .984
schwab_shares

array([11.54744021, 15.96989152, 27.47302554, 20.54526006])

In [57]:
# Shares of etfs to buy without fractional shares
s = np.round(schwab_shares)
s

array([12., 16., 27., 21.])

In [58]:
s*currentPrices

array([1911.36, 1378.24, 1399.14, 1218.  ])

In [59]:
sum(s*currentPrices)

5906.740000000001

In [35]:
y = (s*currentPrices)/sum(s*currentPrices)
# array([0.32650404, 0.22523272, 0.25267487, 0.19558837])

In [36]:
# New weights of portfolio
y * 100

array([32.07754894, 23.13042077, 24.35084987, 20.44118042])

In [37]:
z = y * np.array([0.03,0.035,0.09,0.09])

In [38]:
cost = sum(z)/4
cost

0.01450793480303966

In [39]:
# How much it costs to construct the portfolio
amountInvested * cost/100

0.8595951370800998